In [3]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
import pandas as pd

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('WELFake_Dataset.csv')
df.head(3)

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1


In [6]:
df=df.dropna()

In [7]:
max_words = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'].astype(str))

In [8]:
X = tokenizer.texts_to_sequences(df['text'].astype(str))
X = pad_sequences(X, maxlen=max_sequence_length)

# Prepare target variable
y = df['label'].values

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

vocab_size = len(tokenizer.word_index) + 1  # total words + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(128, return_sequences=False)))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          32044100  
                                                                 
 bidirectional (Bidirection  (None, 256)               234496    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 32278853 (123.13 MB)
Trainable params: 32278853 (123.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)


Epoch 1/5
1789/1789 [==============================] - 631s 349ms/step - loss: 0.2033 - accuracy: 0.9201 - val_loss: 0.1718 - val_accuracy: 0.9400
Epoch 2/5
1789/1789 [==============================] - 654s 366ms/step - loss: 0.1070 - accuracy: 0.9619 - val_loss: 0.1528 - val_accuracy: 0.9439
Epoch 3/5
1789/1789 [==============================] - 706s 395ms/step - loss: 0.0657 - accuracy: 0.9773 - val_loss: 0.1770 - val_accuracy: 0.9407
Epoch 4/5
1789/1789 [==============================] - 695s 388ms/step - loss: 0.0358 - accuracy: 0.9881 - val_loss: 0.1877 - val_accuracy: 0.9461
Epoch 5/5
1789/1789 [==============================] - 731s 409ms/step - loss: 0.0214 - accuracy: 0.9927 - val_loss: 0.2235 - val_accuracy: 0.9448
